In [39]:
# this notebook is for creating three comparison scenarios:
# 1. the model at its initial, unperturbed steady state
# 2. the model immediately (one timestep) after being tilted upstream
# 3. the model after returning to steady state post-tilting
# (4.) (possibly) the model with some component of active tilting

In [40]:
# start by importing libraries
import numpy as np
import matplotlib.pyplot as plt

In [41]:
# SET UP GRID ARRAYS
dx = 1000
x = np.arange(0, 100000, dx)

# set up references to topography
Hstar = 0.1
H = Hstar + np.zeros(len(x))
etab = -H
eta = etab + H

# abrasion coefficients from Attal and Lave 2006 in % per km
beta_ig = np.zeros(len(x))
beta_ig[:50] = 0.000004
beta_sed = np.zeros(len(x))
beta_sed[50:] = 0.000064

# erodibility values
k_ig = np.zeros(len(x))
k_ig[:50] = 0.0001
k_sed = np.zeros(len(x))
k_sed[50:] = 0.001

In [42]:
# write a function that has two bedrock lithologies in the domain, but only one contributing sediment to bedload
def two_lith_one_sed(x,
                     dx,
                     Hstar,
                     H,
                     etab,
                     eta,
                     beta_ig,
                     beta_sed,
                     k_ig,
                     k_sed,
                     c = 1,
                     baselevel_rate = 0.001,
                     num_steps = 850000, 
                     porosity = 0.55):
    
    H[-1] = 0.0
    bedrock_ero = np.zeros(len(x))  # bedrock erosion rate
    sedimentation_rate = np.zeros(len(x))
    total_ero = np.zeros(len(x))
    q = x  # discharge = distance downstream (first node is left edge of 0th cell)
    qs = np.zeros(len(x))  # first node is left edge of 0th cell
    dt = 0.5 * (0.2 * dx * dx / (c*q[-1])) # time-step size

    for i in range(num_steps):
        
        # lower baselevel
        eta[-1] -= baselevel_rate * dt 
        
        # set boundary conditions
        etab[-1] = eta[-1]
        
        # calc slope
        S = -np.diff(eta)/dx
        # S = 0.001
        
        # calculate e factor
        efac = np.exp(- H / Hstar)
        
        # calculate total bedload sed flux and set boundary condition
        qs[1:] = c * q[1:] * S * (1.0 - efac[:-1])
        qs[0] = 0
        
        # calc bedrock erosion from stream power (plucking)
        ero_plucking_ig = efac[:-1] * (k_ig[1:] * q[1:] * S)
        ero_plucking_sed = efac[:-1] * (k_sed[1:] * q[1:] * S)
        
        # calc bedrock erosion from abrasion
        ero_ab_ig = efac[:-1] * (beta_ig[:-1] * qs[:-1])
        ero_ab_sed = efac[:-1] * (beta_sed[:-1] * qs[:-1])
        
        # calc bedrock erosion rate from stream power and abrasion
        bedrock_ero[:-1] = ero_plucking_ig + ero_plucking_sed + ero_ab_ig + ero_ab_sed
        
        # calc change in bedrock elev
        etab[:-1] -= bedrock_ero[:-1] * dt
        
        # calc grain attrition
        atr = 0.000004 * qs[1:]
        
        # calc change in alluvial thickness
        sedimentation_rate[:-1] = -((1 / porosity) * ((np.diff(qs)/dx) + atr - ero_plucking_ig))
        H[:-1] += sedimentation_rate[:-1] * dt
        H[H < 0] = 0
        
        # track total erosion
        total_ero = bedrock_ero - sedimentation_rate
        
        # update elev
        eta[:-1] = etab[:-1] + H[:-1]
        
    return (eta, etab, S, efac, qs, ero_plucking_ig, ero_plucking_sed, ero_ab_ig, ero_ab_sed, bedrock_ero, 
            atr, sedimentation_rate, H, total_ero)

In [43]:
# up first, run our landscape to steady state

(eta_1, 
etab_1,
S_1,
efac_1,
qs_1,
ero_plucking_ig_1, 
ero_plucking_sed_1,
ero_ab_ig_1,
ero_ab_sed_1,
bedrock_ero_1, 
atr_1, 
sedimentation_rate_1, 
H_1, 
total_ero_1) = two_lith_one_sed(x,
                                dx,
                                Hstar,
                                H,
                                etab,
                                eta,
                                beta_ig,
                                beta_sed,
                                k_ig,
                                k_sed)

In [44]:
total_ero_1

array([0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.   ])

In [45]:
# the above cell has just created outputs that have run to steady state 
# (can check this by looking at `total_ero` and seeing that it is the same as the baselevel lowering rate)
# now tilt the topography and bedrock and check out what happens after 1 timestep

In [46]:
upstream_tilt_factor = np.linspace(45, 1, 100)

eta_tilt = eta_1 + upstream_tilt_factor
etab_tilt = etab_1 + upstream_tilt_factor

In [47]:
# now use the tilted bedrock and topography as the inputs to another model run
# use the sediment thickness output at steady state from the previous run as the input for `H`
# (this is because we want to tilt the landscape up, but we do not want to overthicken H)

In [48]:
# # now run for one time step after tectonically tilting the land surface

# (eta_2, 
# etab_2,
# S_2,
# efac_2,
# qs_2,
# ero_plucking_ig_2, 
# ero_plucking_sed_2,
# ero_ab_ig_2,
# ero_ab_sed_2,
# bedrock_ero_2, 
# atr_2, 
# sedimentation_rate_2, 
# H_2, 
# total_ero_2) = two_lith_one_sed(x,
#                                 dx,
#                                 Hstar,
#                                 H_1,
#                                 etab_tilt,
#                                 eta_tilt,
#                                 beta_ig,
#                                 beta_sed,
#                                 k_ig,
#                                 k_sed,
#                                 num_steps = 1)

In [49]:
# # making copies of all the outputs from Run 2 because... idk, something weird is going on
# # and these outputs seem to be affected when I run the model a third time (to steady state)

# eta_2_copy = eta_2.copy()
# etab_2_copy = etab_2.copy()
# S_2_copy = S_2.copy()
# efac_2_copy = efac_2.copy()
# qs_2_copy = qs_2.copy()
# ero_plucking_ig_2_copy = ero_plucking_ig_2.copy()
# ero_plucking_sed_2_copy = ero_plucking_sed_2.copy()
# ero_ab_ig_2_copy = ero_ab_ig_2.copy()
# ero_ab_sed_2_copy = ero_ab_sed_2.copy()
# bedrock_ero_2_copy = bedrock_ero_2.copy()
# atr_2_copy = atr_2.copy()
# sedimentation_rate_2_copy = sedimentation_rate_2.copy()
# H_2_copy = H_2.copy()
# total_ero_2_copy = total_ero_2.copy()

In [50]:
# (eta_4, 
# etab_4,
# S_4,
# efac_4,
# qs_4,
# ero_plucking_ig_4, 
# ero_plucking_sed_4,
# ero_ab_ig_4,
# ero_ab_sed_4,
# bedrock_ero_4, 
# atr_4, 
# sedimentation_rate_4, 
# H_4, 
# total_ero_4) = two_lith_one_sed(x,
#                                 dx,
#                                 Hstar,
#                                 H_1,
#                                 etab_tilt,
#                                 eta_tilt,
#                                 beta_ig,
#                                 beta_sed,
#                                 k_ig,
#                                 k_sed,
#                                 num_steps = 5)

# eta_4_copy = eta_4.copy()
# etab_4_copy = etab_4.copy()
# S_4_copy = S_4.copy()
# efac_4_copy = efac_4.copy()
# qs_4_copy = qs_4.copy()
# ero_plucking_ig_4_copy = ero_plucking_ig_4.copy()
# ero_plucking_sed_4_copy = ero_plucking_sed_4.copy()
# ero_ab_ig_4_copy = ero_ab_ig_4.copy()
# ero_ab_sed_4_copy = ero_ab_sed_4.copy()
# bedrock_ero_4_copy = bedrock_ero_4.copy()
# atr_4_copy = atr_4.copy()
# sedimentation_rate_4_copy = sedimentation_rate_4.copy()
# H_4_copy = H_4.copy()
# total_ero_4_copy = total_ero_4.copy()

In [51]:
# (eta_5, 
# etab_5,
# S_5,
# efac_5,
# qs_5,
# ero_plucking_ig_5, 
# ero_plucking_sed_5,
# ero_ab_ig_5,
# ero_ab_sed_5,
# bedrock_ero_5, 
# atr_5, 
# sedimentation_rate_5, 
# H_5, 
# total_ero_5) = two_lith_one_sed(x,
#                                 dx,
#                                 Hstar,
#                                 H_1,
#                                 etab_tilt,
#                                 eta_tilt,
#                                 beta_ig,
#                                 beta_sed,
#                                 k_ig,
#                                 k_sed,
#                                 num_steps = 25)

# eta_5_copy = eta_5.copy()
# etab_5_copy = etab_5.copy()
# S_5_copy = S_5.copy()
# efac_5_copy = efac_5.copy()
# qs_5_copy = qs_5.copy()
# ero_plucking_ig_5_copy = ero_plucking_ig_5.copy()
# ero_plucking_sed_5_copy = ero_plucking_sed_5.copy()
# ero_ab_ig_5_copy = ero_ab_ig_5.copy()
# ero_ab_sed_5_copy = ero_ab_sed_5.copy()
# bedrock_ero_5_copy = bedrock_ero_5.copy()
# atr_5_copy = atr_5.copy()
# sedimentation_rate_5_copy = sedimentation_rate_5.copy()
# H_5_copy = H_5.copy()
# total_ero_5_copy = total_ero_5.copy()

In [52]:
# (eta_6, 
# etab_6,
# S_6,
# efac_6,
# qs_6,
# ero_plucking_ig_6, 
# ero_plucking_sed_6,
# ero_ab_ig_6,
# ero_ab_sed_6,
# bedrock_ero_6, 
# atr_6, 
# sedimentation_rate_6, 
# H_6, 
# total_ero_6) = two_lith_one_sed(x,
#                                 dx,
#                                 Hstar,
#                                 H_1,
#                                 etab_tilt,
#                                 eta_tilt,
#                                 beta_ig,
#                                 beta_sed,
#                                 k_ig,
#                                 k_sed,
#                                 num_steps = 100)

# eta_6_copy = eta_6.copy()
# etab_6_copy = etab_6.copy()
# S_6_copy = S_6.copy()
# efac_6_copy = efac_6.copy()
# qs_6_copy = qs_6.copy()
# ero_plucking_ig_6_copy = ero_plucking_ig_6.copy()
# ero_plucking_sed_6_copy = ero_plucking_sed_6.copy()
# ero_ab_ig_6_copy = ero_ab_ig_6.copy()
# ero_ab_sed_6_copy = ero_ab_sed_6.copy()
# bedrock_ero_6_copy = bedrock_ero_6.copy()
# atr_6_copy = atr_6.copy()
# sedimentation_rate_6_copy = sedimentation_rate_6.copy()
# H_6_copy = H_6.copy()
# total_ero_6_copy = total_ero_6.copy()

In [53]:
# (eta_7, 
# etab_7,
# S_7,
# efac_7,
# qs_7,
# ero_plucking_ig_7, 
# ero_plucking_sed_7,
# ero_ab_ig_7,
# ero_ab_sed_7,
# bedrock_ero_7, 
# atr_7, 
# sedimentation_rate_7, 
# H_7, 
# total_ero_7) = two_lith_one_sed(x,
#                                 dx,
#                                 Hstar,
#                                 H_1,
#                                 etab_tilt,
#                                 eta_tilt,
#                                 beta_ig,
#                                 beta_sed,
#                                 k_ig,
#                                 k_sed,
#                                 num_steps = 500)

# eta_7_copy = eta_7.copy()
# etab_7_copy = etab_7.copy()
# S_7_copy = S_7.copy()
# efac_7_copy = efac_7.copy()
# qs_7_copy = qs_7.copy()
# ero_plucking_ig_7_copy = ero_plucking_ig_7.copy()
# ero_plucking_sed_7_copy = ero_plucking_sed_7.copy()
# ero_ab_ig_7_copy = ero_ab_ig_7.copy()
# ero_ab_sed_7_copy = ero_ab_sed_7.copy()
# bedrock_ero_7_copy = bedrock_ero_7.copy()
# atr_7_copy = atr_7.copy()
# sedimentation_rate_7_copy = sedimentation_rate_7.copy()
# H_7_copy = H_7.copy()
# total_ero_7_copy = total_ero_7.copy()

In [54]:
# (eta_8, 
# etab_8,
# S_8,
# efac_8,
# qs_8,
# ero_plucking_ig_8, 
# ero_plucking_sed_8,
# ero_ab_ig_8,
# ero_ab_sed_8,
# bedrock_ero_8, 
# atr_8, 
# sedimentation_rate_8, 
# H_8, 
# total_ero_8) = two_lith_one_sed(x,
#                                 dx,
#                                 Hstar,
#                                 H_1,
#                                 etab_tilt,
#                                 eta_tilt,
#                                 beta_ig,
#                                 beta_sed,
#                                 k_ig,
#                                 k_sed,
#                                 num_steps = 1000)

# eta_8_copy = eta_8.copy()
# etab_8_copy = etab_8.copy()
# S_8_copy = S_8.copy()
# efac_8_copy = efac_8.copy()
# qs_8_copy = qs_8.copy()
# ero_plucking_ig_8_copy = ero_plucking_ig_8.copy()
# ero_plucking_sed_8_copy = ero_plucking_sed_8.copy()
# ero_ab_ig_8_copy = ero_ab_ig_8.copy()
# ero_ab_sed_8_copy = ero_ab_sed_8.copy()
# bedrock_ero_8_copy = bedrock_ero_8.copy()
# atr_8_copy = atr_8.copy()
# sedimentation_rate_8_copy = sedimentation_rate_8.copy()
# H_8_copy = H_8.copy()
# total_ero_8_copy = total_ero_8.copy()

In [55]:
# (eta_9, 
# etab_9,
# S_9,
# efac_9,
# qs_9,
# ero_plucking_ig_9, 
# ero_plucking_sed_9,
# ero_ab_ig_9,
# ero_ab_sed_9,
# bedrock_ero_9, 
# atr_9, 
# sedimentation_rate_9, 
# H_9, 
# total_ero_9) = two_lith_one_sed(x,
#                                 dx,
#                                 Hstar,
#                                 H_1,
#                                 etab_tilt,
#                                 eta_tilt,
#                                 beta_ig,
#                                 beta_sed,
#                                 k_ig,
#                                 k_sed,
#                                 num_steps = 5000)

# eta_9_copy = eta_9.copy()
# etab_9_copy = etab_9.copy()
# S_9_copy = S_9.copy()
# efac_9_copy = efac_9.copy()
# qs_9_copy = qs_9.copy()
# ero_plucking_ig_9_copy = ero_plucking_ig_9.copy()
# ero_plucking_sed_9_copy = ero_plucking_sed_9.copy()
# ero_ab_ig_9_copy = ero_ab_ig_9.copy()
# ero_ab_sed_9_copy = ero_ab_sed_9.copy()
# bedrock_ero_9_copy = bedrock_ero_9.copy()
# atr_9_copy = atr_9.copy()
# sedimentation_rate_9_copy = sedimentation_rate_9.copy()
# H_9_copy = H_9.copy()
# total_ero_9_copy = total_ero_9.copy()

In [56]:
# finally, run our tilted landscape back to steady state

(eta_3, 
etab_3,
S_3,
efac_3,
qs_3,
ero_plucking_ig_3, 
ero_plucking_sed_3,
ero_ab_ig_3,
ero_ab_sed_3,
bedrock_ero_3, 
atr_3, 
sedimentation_rate_3, 
H_3, 
total_ero_3) = two_lith_one_sed(x,
                                dx,
                                Hstar,
                                H_1,
                                etab_tilt,
                                eta_tilt,
                                beta_ig,
                                beta_sed,
                                k_ig,
                                k_sed)

In [57]:
total_ero_3

array([0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001,
       0.   ])

In [58]:
# our perturbed landscape is now back at steady state
# so now we just want to look at each of our metrics in comparison and see how things are evolving

In [59]:
# # check out topography
# plt.plot(x, eta_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x, eta_2_copy, label = 'immediate post-tilt')
# plt.plot(x, eta_3, label = 'post-tilt steady state')
# plt.title('topography')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/topographic elevation.png')

In [60]:
# # check out bedrock
# plt.plot(x, etab_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x, etab_2_copy, label = 'immediate post-tilt')
# plt.plot(x, etab_3, label = 'post-tilt steady state')
# plt.title('bedrock')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/bedrock elevation.png')

In [61]:
# # check out slopes
# plt.plot(x[:-1], S_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x[:-1], S_2_copy, label = 'immediate post-tilt')
# plt.plot(x[:-1], S_3, label = 'post-tilt steady state')
# plt.title('slope')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/slopes.png')

In [62]:
# # check out efactor
# plt.plot(x, efac_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x, efac_2_copy, label = 'immediate post-tilt')
# plt.plot(x, efac_3, label = 'post-tilt steady state')
# plt.title('e factor')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/e factor.png')

In [63]:
# # check out qs
# plt.plot(x, qs_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x, qs_2_copy, label = 'immediate post-tilt')
# plt.plot(x, qs_3, label = 'post-tilt steady state')
# plt.title('sediment flux')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/sediment flux.png')

In [64]:
# # check out igneous plucking
# plt.plot(x[:-1], ero_plucking_ig_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x[:-1], ero_plucking_ig_2_copy, label = 'immediate post-tilt')
# plt.plot(x[:-1], ero_plucking_ig_3, label = 'post-tilt steady state')
# plt.title('igneous plucking rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/igneous plucking rate.png')

In [65]:
# # check out sedimentary plucking
# plt.plot(x[:-1], ero_plucking_sed_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x[:-1], ero_plucking_sed_2_copy, label = 'immediate post-tilt')
# plt.plot(x[:-1], ero_plucking_sed_3, label = 'post-tilt steady state')
# plt.title('sedimentary plucking rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/sedimentary plucking rate.png')

In [66]:
# # check out igneous abrasion
# plt.plot(x[:-1], ero_ab_ig_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x[:-1], ero_ab_ig_2_copy, label = 'immediate post-tilt')
# plt.plot(x[:-1], ero_ab_ig_3, label = 'post-tilt steady state')
# plt.title('igneous abrasion rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/igneous abrasion rate.png')

In [67]:
# # check out sedimentary abrasion
# plt.plot(x[:-1], ero_ab_sed_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x[:-1], ero_ab_sed_2_copy, label = 'immediate post-tilt')
# plt.plot(x[:-1], ero_ab_sed_3, label = 'post-tilt steady state')
# plt.title('sedimentary abrasion rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/sedimentary abrasion rate.png')

In [68]:
# # check out bedrock erosion
# plt.plot(x, bedrock_ero_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x, bedrock_ero_2_copy, label = 'immediate post-tilt')
# plt.plot(x, bedrock_ero_3, label = 'post-tilt steady state')
# plt.title('bedrock erosion rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/bedrock erosion rate.png')

In [69]:
# # check out attrition rate
# plt.plot(x[:-1], atr_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x[:-1], atr_2_copy, label = 'immediate post-tilt')
# plt.plot(x[:-1], atr_3, label = 'post-tilt steady state')
# plt.title('grain attrition rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/grain attrition rate.png')

In [70]:
# # check out sedimentation rate
# plt.plot(x, sedimentation_rate_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x, sedimentation_rate_2_copy, label = 'immediate post-tilt')
# plt.plot(x, sedimentation_rate_3, label = 'post-tilt steady state')
# plt.title('sedimentation rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/sedimentation rate.png')

In [71]:
# # check out sediment thickness
# plt.plot(x, H_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x, H_2_copy, label = 'immediate post-tilt')
# plt.plot(x, H_3, label = 'post-tilt steady state')
# plt.title('sediment thickness')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/sediment thickness.png')

In [72]:
# # check out total erosion rate
# plt.plot(x, total_ero_1, label = 'initial steady state', linestyle = '--', linewidth = 3)
# plt.plot(x, total_ero_2_copy, label = 'immediate post-tilt')
# plt.plot(x, total_ero_3, label = 'post-tilt steady state')
# plt.title('total erosion rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/total erosion rate.png')

In [73]:
# now I want to check out just a few metrics in their transient state, 
# as they relax from perturbed back to steady state
# I will look at:
# total erosion rate
# sediment thickness
# sedimentation rate
# bedrock erosion rate
# sedimentary plucking rate
# igneous plucking rate
# sediment flux
# e factor
# slope
# I will look at 1 (2_copy), 5 (4), 25 (5), 100 (6), 500 (7), 1000 (8), and 5000 (9) timesteps

In [74]:
# # transient sediment thickness

# plt.figure(figsize = (10, 6))

# plt.plot(x, H_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x, H_2_copy, label = 'immediate post-tilt')
# # plt.plot(x, H_4_copy, label = '5 timesteps')
# # plt.plot(x, H_5_copy, label = '25 timesteps')
# # plt.plot(x, H_6_copy, label = '100 timesteps')
# # plt.plot(x, H_7_copy, label = '500 timesteps')
# # plt.plot(x, H_8_copy, label = '1000 timesteps')
# # plt.plot(x, H_9_copy, label = '5000 timesteps')
# plt.plot(x, H_3, label = 'post-tilt steady state')
# plt.title('sediment thickness')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient sediment thickness.png')

In [75]:
# # transient total erosion rates

# plt.figure(figsize = (10, 6))

# plt.plot(x, total_ero_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x, total_ero_2_copy, label = 'immediate post-tilt')
# # plt.plot(x, total_ero_4_copy, label = '5 timesteps')
# # plt.plot(x, total_ero_5_copy, label = '25 timesteps')
# # plt.plot(x, total_ero_6_copy, label = '100 timesteps')
# # plt.plot(x, total_ero_7_copy, label = '500 timesteps')
# # plt.plot(x, total_ero_8_copy, label = '1000 timesteps')
# # plt.plot(x, total_ero_9_copy, label = '5000 timesteps')
# plt.plot(x, total_ero_3, label = 'post-tilt steady state')
# plt.title('total erosion rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient total erosion rate.png')

In [76]:
# plt.figure(figsize = (10, 6))

# plt.plot(x, sedimentation_rate_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x, sedimentation_rate_2_copy, label = 'immediate post-tilt')
# # plt.plot(x, sedimentation_rate_4_copy, label = '5 timesteps')
# # plt.plot(x, sedimentation_rate_5_copy, label = '25 timesteps')
# # plt.plot(x, sedimentation_rate_6_copy, label = '100 timesteps')
# # plt.plot(x, sedimentation_rate_7_copy, label = '500 timesteps')
# # plt.plot(x, sedimentation_rate_8_copy, label = '1000 timesteps')
# # plt.plot(x, sedimentation_rate_9_copy, label = '5000 timesteps')
# plt.plot(x, sedimentation_rate_3, label = 'post-tilt steady state')
# plt.title('sedimentation rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient sedimentation rate.png')

In [77]:
# # transient bedrock erosion rate

# plt.figure(figsize = (10, 6))

# plt.plot(x, bedrock_ero_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x, bedrock_ero_2_copy, label = 'immediate post-tilt')
# # plt.plot(x, bedrock_ero_4_copy, label = '5 timesteps')
# # plt.plot(x, bedrock_ero_5_copy, label = '25 timesteps')
# # plt.plot(x, bedrock_ero_6_copy, label = '100 timesteps')
# # plt.plot(x, bedrock_ero_7_copy, label = '500 timesteps')
# # plt.plot(x, bedrock_ero_8_copy, label = '1000 timesteps')
# # plt.plot(x, bedrock_ero_9_copy, label = '5000 timesteps')
# plt.plot(x, bedrock_ero_3, label = 'post-tilt steady state')
# plt.title('bedrock erosion rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient bedrock erosion rate.png')

In [78]:
# # transient sedimentary plucking rate

# plt.figure(figsize = (10, 6))

# plt.plot(x[:-1], ero_plucking_sed_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x[:-1], ero_plucking_sed_2_copy, label = 'immediate post-tilt')
# # plt.plot(x[:-1], ero_plucking_sed_4_copy, label = '5 timesteps')
# # plt.plot(x[:-1], ero_plucking_sed_5_copy, label = '25 timesteps')
# # plt.plot(x[:-1], ero_plucking_sed_6_copy, label = '100 timesteps')
# # plt.plot(x[:-1], ero_plucking_sed_7_copy, label = '500 timesteps')
# # plt.plot(x[:-1], ero_plucking_sed_8_copy, label = '1000 timesteps')
# # plt.plot(x[:-1], ero_plucking_sed_9_copy, label = '5000 timesteps')
# plt.plot(x[:-1], ero_plucking_sed_3, label = 'post-tilt steady state')
# plt.title('sedimentary plucking rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient sedimentary plucking rate.png')

In [79]:
# # transient igneous plucking rate

# plt.figure(figsize = (10, 6))

# plt.plot(x[:-1], ero_plucking_ig_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x[:-1], ero_plucking_ig_2_copy, label = 'immediate post-tilt')
# # plt.plot(x[:-1], ero_plucking_ig_4_copy, label = '5 timesteps')
# # plt.plot(x[:-1], ero_plucking_ig_5_copy, label = '25 timesteps')
# # plt.plot(x[:-1], ero_plucking_ig_6_copy, label = '100 timesteps')
# # plt.plot(x[:-1], ero_plucking_ig_7_copy, label = '500 timesteps')
# # plt.plot(x[:-1], ero_plucking_ig_8_copy, label = '1000 timesteps')
# # plt.plot(x[:-1], ero_plucking_ig_9_copy, label = '5000 timesteps')
# plt.plot(x[:-1], ero_plucking_ig_3, label = 'post-tilt steady state')
# plt.title('igneous plucking rate')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient igneous plucking rate.png')

In [80]:
# # transient sediment flux

# plt.figure(figsize = (10, 6))

# plt.plot(x, qs_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x, qs_2_copy, label = 'immediate post-tilt')
# # plt.plot(x, qs_4_copy, label = '5 timesteps')
# # plt.plot(x, qs_5_copy, label = '25 timesteps')
# # plt.plot(x, qs_6_copy, label = '100 timesteps')
# # plt.plot(x, qs_7_copy, label = '500 timesteps')
# # plt.plot(x, qs_8_copy, label = '1000 timesteps')
# # plt.plot(x, qs_9_copy, label = '5000 timesteps')
# plt.plot(x, qs_3, label = 'post-tilt steady state')
# plt.title('sediment flux')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient sed flux.png')

In [81]:
# # transient e factor

# plt.figure(figsize = (10, 6))

# plt.plot(x, efac_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x, efac_2_copy, label = 'immediate post-tilt')
# # plt.plot(x, efac_4_copy, label = '5 timesteps')
# # plt.plot(x, efac_5_copy, label = '25 timesteps')
# # plt.plot(x, efac_6_copy, label = '100 timesteps')
# # plt.plot(x, efac_7_copy, label = '500 timesteps')
# # plt.plot(x, efac_8_copy, label = '1000 timesteps')
# # plt.plot(x, efac_9_copy, label = '5000 timesteps')
# plt.plot(x, efac_3, label = 'post-tilt steady state')
# plt.title('e factor')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient e factor.png')

In [82]:
# # transient slope

# plt.figure(figsize = (10, 6))

# plt.plot(x[:-1], S_1, label = 'steady state', linestyle = '--', linewidth = 3)
# # plt.plot(x[:-1], S_2_copy, label = 'immediate post-tilt')
# # plt.plot(x[:-1], S_4_copy, label = '5 timesteps')
# # plt.plot(x[:-1], S_5_copy, label = '25 timesteps')
# # plt.plot(x[:-1], S_6_copy, label = '100 timesteps')
# # plt.plot(x[:-1], S_7_copy, label = '500 timesteps')
# # plt.plot(x[:-1], S_8_copy, label = '1000 timesteps')
# # plt.plot(x[:-1], S_9_copy, label = '5000 timesteps')
# plt.plot(x[:-1], S_3, label = 'post-tilt steady state')
# plt.title('slope')
# plt.legend()

# # plt.savefig('/Users/vanessa/Desktop/FA 2021/outputs for 9_15_21 meeting/transient slope.png')